# Working with Rows and Ordered Collections in `pctheory`

`pctheory` uses standard Python lists to work with rows and row fragments. Row fragments are called "pcsegs" which stands for "pitch class segments."

In [2]:
from pctheory import pitch, pcseg, transformations

# A pitch-class segment (pcseg)
frag1 = [pitch.PitchClass(1), pitch.PitchClass(5), pitch.PitchClass(7), pitch.PitchClass(4)]
# Another way to make the same thing:
frag1 = pcseg.make_pcseg12(1, 5, 7, 4)
print(frag1)

row1 = pcseg.make_pcseg12(1, 5, 7, 4, 0, 9, 11, 8, 10, 2, 3, 6)
print(row1)

[1, 5, 7, 4]
[1, 5, 7, 4, 0, 9, B, 8, A, 2, 3, 6]


You can, of course, make a twelve-tone row by listing all twelve pitch-classes in order. But you can also generate rows using functionality in `pctheory.pcseg`. For example, say you just want a random twelve-tone row:

In [3]:
row2 = pcseg.generate_random_pcseg12(12, True)
print(row2)

[9, 5, 2, 6, 8, B, 4, 7, A, 1, 0, 3]


There are special rows in twelve-tone theory, such as all-interval rows and all-trichord rows. `pctheory` can generate random rows with these characteristics. An all-interval rows contains each of the 11 ordered pitch-class intervals exactly once. An all-trichord row contains each of the 12 trichordal set-classes as imbricated fragments. This means that if you take any chunk of 3 adjacent pitch-classes in the row, you will find each of the 12 trichordal set-classes exactly once. (You have to connect the first and last pitch-classes in the row to make this work.)

In [4]:
row3 = pcseg.generate_random_all_interval_row()
row4 = pcseg.generate_random_all_trichord_row()
print(row3)
print(row4)

[0, 3, 4, 8, A, 9, 5, 2, 7, 1, B, 6]
[0, 4, 2, 7, 8, A, 9, 5, 1, B, 6, 3]


We can prove that `row4` is an all-trichord row:

In [5]:
scs = pcseg.imb_n(row4, 3)
print(scs)

[(3-6)[024], (3-7)[025], (3-5)[016], (3-2)[013], (3-1)[012], (3-4)[015], (3-12)[048], (3-8)[026], (3-9)[027], (3-11)[037]]


This proves that there are at least 10 trichordal set-classes in there. To get all 12, we need to add the first two pitches of the row at the end. Now the list of imbricated trichordal set-classes contains all twelve trichordal set-classes.

In [6]:
row4a = row4 + row4[:2]
print(row4a)
scs = pcseg.imb_n(row4a, 3)
print(scs)

[0, 4, 2, 7, 8, A, 9, 5, 1, B, 6, 3, 0, 4]
[(3-6)[024], (3-7)[025], (3-5)[016], (3-2)[013], (3-1)[012], (3-4)[015], (3-12)[048], (3-8)[026], (3-9)[027], (3-11)[037], (3-10)[036], (3-3)[014]]


If you'd like to make a twelve-tone matrix, here is how you do that.

In [7]:
mx = pcseg.TwelveToneMatrix(row4)
print(mx)

     T0I   T4I   T2I   T7I   T8I   T10I  T9I   T5I   T1I   T11I  T6I   T3I   
T0    0     4     2     7     8     A     9     5     1     B     6     3    T0R
T8    8     0     A     3     4     6     5     1     9     7     2     B    T8R
T10   A     2     0     5     6     8     7     3     B     9     4     1   T10R
T5    5     9     7     0     1     3     2     A     6     4     B     8    T5R
T4    4     8     6     B     0     2     1     9     5     3     A     7    T4R
T2    2     6     4     9     A     0     B     7     3     1     8     5    T2R
T3    3     7     5     A     B     1     0     8     4     2     9     6    T3R
T7    7     B     9     2     3     5     4     0     8     6     1     A    T7R
T11   B     3     1     6     7     9     8     4     0     A     5     2   T11R
T1    1     5     3     8     9     B     A     6     2     0     7     4    T1R
T6    6     A     8     1     2     4     3     B     7     5     0     9    T6R
T9    9     1     B     4     5

However, you might not actually need to generate the matrix because it is easy to transform the row and search transformed row-forms using `pctheory`. For example, to transform a row:

In [8]:
ro = transformations.get_otos12()
print(ro["T5R"](row4))
print(ro["T4I"](row4))
print(ro["T9"](row4))
print(ro["T9RI"](row4))

[8, B, 4, 6, A, 2, 3, 1, 0, 7, 9, 5]
[4, 0, 2, 9, 8, 6, 7, B, 3, 5, A, 1]
[9, 1, B, 4, 5, 7, 6, 2, A, 8, 3, 0]
[6, 3, A, 8, 4, 0, B, 1, 2, 7, 5, 9]


And to find a row form with a specific ordered succession of pitch-classes (note that you might need to edit the row fragment to find any results):

In [9]:
row_frag = pcseg.make_pcseg12(5, 9, 0)
row_class = pcseg.get_row_class(row4)
rows = pcseg.adjacent_search(row_frag, row_class)
for row in rows:
    tx = transformations.find_otos(row4, row)
    print(f"{tx}: {row}")

If you're a good twelve-tone composer, you probably want to study your row before using it. We'll revert back to `row1` that we generated, and look at what imbricated trichords it contains.

In [10]:
imb1 = pcseg.imb_n(row1, 3)
print(imb1)

[(3-8)[026], (3-2)[013], (3-11)[037], (3-11)[037], (3-2)[013], (3-2)[013], (3-2)[013], (3-8)[026], (3-4)[015], (3-3)[014]]


It is very likely that this randomly generated row contains at least one trichord more than once. This means that you can bring out that trichord in a composition. You can do the same for tetrachords and chords of other sizes. Note that the all-trichord row that we generated earlier has each trichord exactly once (under rotation), which lets you achieve great harmonic diversity.

The next thing we'll do is generate an invariance matrix. This tells us how a row behaves under transformation.

In [11]:
imx = pcseg.InvarianceMatrix(row1, row1, "T")
imx.print({pitch.PitchClass(5)})

    1 5 7 4 0 9 B 8 A 2 3 6 
   ------------------------
B |                       5 
7 |                 5       
5 |         5               
8 |           5             
0 |   5                     
3 |                   5     
1 |       5                 
4 | 5                       
2 |                     5   
A |     5                   
9 |               5         
6 |             5           


The invariance matrix tells us what happens to the order of the pitch classes in the row when we (in this case) transpose by 5. If you print out the row and its $T_5$ transformation, you'll see that the row has been shuffled as the matrix predicted. The invariance matrix helps us see what successions of pitch classes are preserved under transformation.

In [12]:
print(row1)
print(ro["T5"].transform(row1))

[1, 5, 7, 4, 0, 9, B, 8, A, 2, 3, 6]
[6, A, 0, 9, 5, 2, 4, 1, 3, 7, 8, B]


Finally, we'll generate a rotational array in the style of Stravinsky.

In [13]:
from pctheory import pcarray
frag = pcseg.make_pcseg12(5, 9, 8, 1, 4, 0)
arr = pcarray.RotationalArray(frag)
print(arr)

0 4 3 8 B 7 
0 B 4 7 3 8 
0 5 8 4 9 1 
0 3 B 4 8 7 
0 8 1 5 4 9 
0 5 9 8 1 4 
